# Insight week 6, code challenge


The goal of this challenge is to sort read a file which contains one line, for which we want to sort the space delimited contents.   Integers-like strings must be sorted as integers and the rest must be sorted as strings.  Moreover, if an int/string appears in the n-th entry in this list an int/string must appear there in the sorted list.  Finally, the string can be filled with arbitray ascii which is not in the alphanumeric we want.  Here is a comprehensible example:

<code>20 cat bi?rd 12 do@g</code>
---

must read

<code>12 bird cat 20 dog</code>
---

---


## Algorithm

My approach is fairly simple:

- Read in string from file
- Use regular expressions (actually string.translate) to remove all ascii besides alphanumeric, spaces, and dashes
- Split string into a list
- Go through the list, record integers in one array and alphabetic in another (be careful with dashes)
- As the list is traverse, also keep track whether the element is a int or str using a dictionary
- Sort the integer and string arrays, using python's sorted function (Timsort)
- Go through the list again and replace the elements with the sorted ones, using are dict to decide int or str
- Join the list to a string using space delimitation, write to file.


---


## Run the code.

###First, import the sorter, and run unit tests

In [10]:
from listSorting import SortList
from test import TestSorterMethods
import unittest, sys

suite = unittest.TestLoader().loadTestsFromTestCase(TestSorterMethods)
unittest.TextTestRunner(verbosity=1,stream=sys.stderr).run(suite)

....
----------------------------------------------------------------------
Ran 4 tests in 0.021s

OK


<unittest.runner.TextTestResult run=4 errors=0 failures=0>



### An example test

I have written a string generator (string_gen in test.py) which simulates the problem, lets import and have a look at a test string.

In [2]:
from test import string_gen
test, true, _, _, _ = string_gen(20, seed=123)
print test

�2�5`�37�00 �1�5597 cCIMrtQwHGXVjG 434420 GVztoKGqeXdcuNc WhPJCMHvEBIHmOdQ �5���847/�76 alIkwn 
K�٣p�R�BS/E��g��T=Z�#Aqg&oNlhbRL uXmsrbZSPWBXwddlvzN -663129 IdldZEgjxo tgmBMrU JJbVTqfOTwU 36377 OzTXaJDbteEhDMbm �-��1Ͽ9065 �`�9��16046 405341 �4&63�661




Cool, our test string looks like a mess!  Lets print the desired result, and the result of running SortList

In [9]:
print 'True sorted list:\n', true
sorter = SortList()
sorter.sort_line(test)
print '\nMy sorted list:\n', sorter.result

True sorted list:
-663129 -19065 alIkwn 15597 cCIMrtQwHGXVjG GVztoKGqeXdcuNc 36377 IdldZEgjxo JJbVTqfOTwU KpRBSEgTZAqgoNlhbRL 253700 OzTXaJDbteEhDMbm tgmBMrU uXmsrbZSPWBXwddlvzN 405341 WhPJCMHvEBIHmOdQ 434420 463661 584776 916046

My sorted list:
-663129 -19065 alIkwn 15597 cCIMrtQwHGXVjG GVztoKGqeXdcuNc 36377 IdldZEgjxo JJbVTqfOTwU KpRBSEgTZAqgoNlhbRL 253700 OzTXaJDbteEhDMbm tgmBMrU uXmsrbZSPWBXwddlvzN 405341 WhPJCMHvEBIHmOdQ 434420 463661 584776 916046


Awesome, the desired result is achieved!

## Lets look at the run time of the code.

###Lets define a timing function, run while doubling the number of entries

In [47]:
from time import time

def timer(sorter, Nvals, repeat=100, seed=1234):
    """Compute the average run time for Nvals.  Use this (instead 
    of e.g., timeit) so that the time of the text/file generation
    is not included.
    
    Args:
        sorter (class): Class for sorting string.
        Nvals (list): List of ints which are the number of words
                      or ints in the string.
        repeat (Optional[int]): Number of repeated trials to
                                average over
        seed (Optional[int]): Seed for random number generator
                                
    Returns:
        avg_times (list): Average runtime for Nvals
    """
    avg_times = []
    for i, N in enumerate(Nvals):
        tot_time = 0.
        # generate the test string and write to file
        test, _, _, _, _ = string_gen(N, seed=seed)
        sorter.result = test
        sorter._write('test_list.txt')
        for j in range(repeat):    
            # only time the actual call
            t = time()
            sorter.__init__('./test_list.txt', './test_result.txt')
            tot_time += (time() - t)
            seed += 1
        avg_times.append(tot_time / repeat)
    return avg_times

In [48]:
N = [12500, 25000, 50000, 100000, 200000]
avg_times = timer(SortList(), N)

In [49]:
print avg_times

[0.02755892038345337, 0.05881011486053467, 0.123423752784729, 0.272075879573822, 0.5438107132911683]


### Lets compute how it scales and compare to big-o

The time complexity of the sorting algorithm is set by the steps with the largest complexity.  In this case it is the time complexity of Timsort, which is 

$$O(n\log(n))$$.

Lets see if our experiment confirms this.  We have computed the average runtime for 12.5k, 25k, 50k, 100k, and 200k entries. Since we expect the algorithm to scale as above, we should expect the ratio of the times to be:

In [50]:
def expected_scaling(N1, N2):
    """Expected scaling given samples sizes N1 and N2, given O = NlogN

    Args:
        N1 (int): Number of samples in first run.
        N2 (int): Number of samples in second run.
        
    Returns:
        (list): Ratios of run times.
    """
    return (1. * N2 * np.log(N2)) / (N1 * np.log(N1))
    
print 'Empirical run time ratios over 100 trials:\n', [avg_times[i] / avg_times[i - 1] for i in range(1, 4)]
print '\nExpected scaling for NlogN runtime:\n',[expected_scaling(N[i-1], N[i]) for i in range(1, 4)]

Empirical run time ratios over 100 trials:
[2.1339774578341175, 2.0986823963432557, 2.2044045285867004]

Expected scaling for NlogN runtime:
[2.1469546534867416, 2.1368959081162533, 2.1281259490425364]


While not exact, our run times agree pretty well!

## Alternatives, conclusion

Since python's timsort is $$O(n\log(n))$$ there are a few time-complexity-equivalent ways to approach the sorting.  Merge sort (which is part of timsort) and quick sort are two options.  Other generic sorts (selection, insertion, ...) are $$O(n^2)$$ so will not improve the situation.  In principle if the number of duplicates are very high, a hash table may be useful to get significant speedups and could approach $$O(n)$$.  Future versions of this code could explore this, and perhaps implement a check to determine the appropriate approach.